In [1]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json

In [2]:
def scrape(list_of_cols,save_as):
    url = 'https://coinalyze.net/bitcoin/basis/'
    timestamp = time.time()
    current_btc_price = get_btc_value()
    # Request content from web page
    page = requests.get(url)

    # Parse the html content
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', {'class': ''})

    table_headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]

    # Extract the table data
    table_data = [[td.get_text().strip() for td in tr.find_all("td")]
                for tr in table.find_all("tr")[1:]]

    # Create the dataframe
    df = pd.DataFrame(table_data, columns=table_headings)

    if save_as == 'json':
        #Save as json
        out_list = df[list_of_cols].to_dict('records')

        return {
            'timestamp':timestamp,
            'current_btc_price':current_btc_price,
            'data':out_list
        }
    else:
        #Save as csv
        df['timestamp'] = timestamp
        df['current_btc_price'] = current_btc_price

        return df
def get_btc_value():
  
    # defining key/request url
    key = "https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
    
    # requesting data from url
    data = requests.get(key)  
    data = data.json()
    return float(data['price'])

In [3]:
#Constants
list_of_cols = ["Exchange","Contract","Price","OI","Basis"] #List of columns to be included in the output
every_n_mins = 5 # Waiting time (in minutes)
out_file = "out" #Name of the output file
save_as = 'csv' #or csv # switch

In [4]:
out_list = []
out_df = pd.DataFrame()
while True:
    out_data = scrape(list_of_cols=list_of_cols,save_as=save_as)
    if save_as == 'json':
        print(f"Timestamp: {out_data['timestamp']}")
        out_list.append(out_data)

        with open(f"{out_file}.json", 'w') as f:
            json.dump(out_list, f)
    else:
        #If csv
        print(f"Timestamp: {out_data['timestamp'][0]}")
        out_df = pd.concat([out_df,out_data])

        #Save
        out_df.to_csv(f'{out_file}.csv',index=False)
    time.sleep(every_n_mins*60)

Timestamp: 1673965114.424547


KeyboardInterrupt: 